<a href="https://colab.research.google.com/github/mmtaha/Projeto_Python_Busca_Cliente_Produto_Empresa/blob/main/Projeto_Oracle_Busca_Empresa_Produto_Oppty_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BLOCO 1: IMPORTAÇÕES E CONFIGURAÇÕES INICIAIS

# Se estiver em Jupyter e precisar instalar dependências, descomente:
!pip install pyxlsb unidecode rapidfuzz scikit-learn xlsxwriter --quiet

import pandas as pd
import re
import unidecode
from functools import lru_cache
from IPython.display import display
from collections import Counter

# ==== Configurações iniciais ====
try:
    from rapidfuzz import fuzz, process, distance
    USING = "rapidfuzz"
    print("ℹUsando rapidfuzz")
except Exception:
    from fuzzywuzzy import fuzz, process
    USING = "fuzzywuzzy"
    distance = None
    print("ℹUsando fuzzywuzzy")

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    HAVE_SKLEARN = True
    print("ℹCosine check habilitado")
except Exception:
    HAVE_SKLEARN = False
    print("ℹscikit-learn não disponível")

# ==== Parâmetros ====
MATCH_THRESHOLD = 95
JW_MIN = 90
COSINE_MIN = 0.95
TOPN_PER_ROW = 40

# ==== PALAVRAS-CHAVE PRIORITÁRIAS ESPECÍFICAS PEPSICO ====
PALAVRAS_CHAVE_PRIORITARIAS = [
    # Nomes exatos da PepsiCo
    "pepsico", "pepsi-cola", "pepsicola", "pepsi cola",
    # Marcas principais da PepsiCo
    "sabritas", "gamesa", "quaker", "gatorade", "ruffles",
    "cheetos", "doritos", "lays", "frito-lay", "mirinda",
    "7up", "sonrics", "emmanuel", "barcel", "tostitos",
    "tropicana", "manaos", "alvalle", "happy",
    "mabel", "elma chips", "evercrisp", "fruko", "holy",
    "kero", "matutino", "mayco", "natural hit",
    "natu", "paps", "paseo", "pimpollo", "rancrisp",
    "ricky", "ritmo", "rosquinhas", "salmas", "salticas",
    "santa maria", "santiveri", "smacks", "tosty", "tranquitas",
    "trident", "trocipollo", "twist", "vive 100", "yogu yogu",
    # Subsidiárias operacionais
    "alimentos del istmo", "alimesa", "comercializadora nacional",
    "corporativo internacional mexicano", "latin american holdings",
    "snacks america latina", "bebidas sudamerica", "inversiones borneo",
    "inversiones pfi", "servicios gbf", "servicios gflg", "servicios syc",
    "pepsico alimentos", "pepsico alimentos mexico", "pepsico argentina",
    "pepsico brasil", "pepsico chile", "pepsico colombia", "pepsico mexico",
    "pepsico venezuela", "pepsico andina", "pepsico latina", "pepsico caribe",
    "productos pepsi", "bebidas pepsi", "refrescos pepsi", "jugos pepsi",
    "snacks pepsi", "alimentos pepsi"
]

# ==== PALAVRAS PARA EXCLUSÃO CONTEXTUAL RIGOROSA ====
PALAVRAS_EXCLUSAO_CONTEXTUAL = [
    "barcelos", "barcelona", "barclays", "tropicana hotel", "tropicana resort",
    "tropicana casino", "starbucks store", "starbucks cafe", "starbucks coffee",
    "hotel", "resort", "casino", "restaurant", "cafe", "coffee", "university",
    "school", "hospital", "clinic", "government", "municipal", "federal",
    "estadual", "health", "medical", "insurance", "bank", "banco", "finance",
    "investment", "real estate", "construction", "engineering", "consulting",
    "law", "legal", "accounting", "education", "research", "technology",
    "software", "hardware", "it", "telecom", "media", "entertainment", "sports",
    "travel", "tourism", "transport", "logistics", "energy", "power", "water",
    "utility", "pharma", "pharmaceutical", "medical", "healthcare", "insurance"
]

# ==== FUNÇÕES DE NORMALIZAÇÃO ====
def base_normalize(text: str) -> str:
    if pd.isna(text):
        return ""
    s = unidecode.unidecode(str(text).lower())
    s = re.sub(r"[\(\[{].*?[\)\]}]", " ", s)
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# CORREÇÃO AQUI - padrão regex simplificado
SEP = r"[\s.,/\-_(){};:]+"

def flex_pat(phrase: str) -> str:
    tokens = phrase.strip().split()
    return r"\b" + (SEP.join([re.escape(t) for t in tokens])) + r"\b"

# ==== SUFIXOS CORPORATIVOS ====
CORPORATE_SUFFIXES = [
    "ltda", "limitada", "me", "eireli", "s/a", "sa", "s.a", "s a", "cia", "companhia",
    "empresa", "comercio", "comércio", "servicos", "serviços", "industria", "industrial",
    "distribuidora", "sas", "s.a.s", "s a s", "srl", "s.r.l", "s r l", "sa de cv",
    "s.a. de c.v", "s a de c v", "eurl", "e.u.r.l", "sc", "s.c", "sp", "s.p", "scs",
    "s.c.s", "senc", "s.en.c", "sarl", "s.a.r.l", "inc", "incorporated", "corp",
    "corporation", "ltd", "limited", "llc", "lp", "l.p", "pllc", "llp", "plc", "pty",
    "pte", "s a", "gp", "companies", "pc", "p.c", "lc", "l.c", "lllp", "gmbh", "ag",
    "kg", "kgaa", "ggmbh", "sarl", "sas", "eurl", "snc", "s.n.c", "scs", "sca", "s.c.a",
    "srl", "spa", "s.p.a", "snc", "sas", "bv", "nv", "vof", "cv", "ab", "oy", "as",
    "aps", "kg", "zrt", "nyrt", "co ltd", "company limited", "pt", "pvt ltd",
    "private limited", "kk", "yk", "gk", "sdn bhd", "bhd", "cc", "pty ltd", "shma",
    "llc", "pjsc", "asc", "nl", "group", "international", "worldwide", "global",
    "ventures", "enterprises"
]

NON_DISTINCTIVE_WORDS = [
    "group", "grupo", "grup", "gr", "gr.", "international", "internacional", "intl",
    "global", "worldwide", "world", "companies", "enterprises", "ventures", "partners",
    "partnership", "management", "capital", "investment", "investments", "fund", "trust",
    "services", "solutions", "technologies", "technology", "tec", "tech", "tecno",
    "systems", "system", "industries", "industry", "industrial", "comercial", "commercial",
    "financeiro", "financial", "retail", "varejo", "atacado", "wholesale", "manufacturing",
    "manufactura", "inversiones", "Ltd.", "Limited", "S.R.L.", "SRL", "LLC", "Inc.",
    "N.V.", "Logistica", "Holding", "Holdings", "LOGISTICS", "Servicios", "BANCO",
    "S/A", "Group", "Comercializadora", "Seguros", "health", "healthcare", "pharma",
    "pharmaceutical", "medical", "med", "energy", "energia", "power", "utilities",
    "utility", "sports", "deportivo", "entertainment", "media", "communications",
    "telecom", "construction", "construção", "engineering", "engenharia", "imobiliario",
    "property", "properties", "hotel", "hospitality", "travel", "viagens", "tour",
    "tourism", "auto", "automotive", "car", "vehicle", "motor", "insurance", "seguros",
    "bank", "banco", "financial", "finanças", "asset", "assets", "security", "segurança",
    "software", "hardware", "it", "information technology", "data", "digital", "online",
    "web", "net", "network", "cloud", "consulting", "consultoria", "advisory", "law",
    "legal", "accounting", "contabilidade", "audit", "auditoria", "education", "educação",
    "school", "university", "research", "pesquisa", "development", "desenvolvimento",
    "design", "planning", "planejamento", "marketing", "advertising", "publicidade",
    "sales", "vendas", "support", "suporte", "care", "cuidado", "home", "casa", "office",
    "escritorio", "business", "negocio", "corp", "company", "co", "com", "and", "e", "y",
    "of", "de", "do", "da", "das", "dos", "the", "el", "la", "los", "las", "le", "les",
    "en", "et", "und", "&"
]

_base_suffix_phrases = CORPORATE_SUFFIXES + NON_DISTINCTIVE_WORDS
_suffix_regexes = [re.compile(flex_pat(p), flags=re.IGNORECASE) for p in _base_suffix_phrases]

_tail_suffix_tokens = (
    {p.strip().lower() for p in _base_suffix_phrases if p and p.strip()} |
    {"cv", "rl", "c", "v", "de", "por", "a", "s", ".a", "b", "d", ".d", "m", "l", "k",
     "j", "i", "h", "e", "f", "n", "g", "o", "p", "q", "r", "t", "u", "x", "y", "z", "w"}
)
_SUFFIX_SET_NORM = {p.strip().lower() for p in _base_suffix_phrases if p and p.strip()}

# ==== FUNÇÕES DE LIMPEZA ====
def strip_suffixes(s: str) -> str:
    if pd.isna(s):
        return ""
    out = str(s)
    for rgx in _suffix_regexes:
        out = rgx.sub(" ", out)
    out = re.sub(r"\s+", " ", out).strip()
    words = out.split()
    cut = 0
    while words and cut < 5 and words[-1].lower() in _tail_suffix_tokens:
        words.pop()
        cut += 1
    return " ".join(words).strip()

def remove_suffix_tokens(text: str) -> str:
    if not text:
        return ""
    words = text.split()
    filtered = []
    for w in words:
        wl = w.lower().strip(".,;")
        if wl in _tail_suffix_tokens or wl in _SUFFIX_SET_NORM:
            continue
        filtered.append(w)
    return " ".join(filtered)

@lru_cache(maxsize=8)
def _compile_country_regexes(country_tuple):
    norms = []
    for c in country_tuple:
        cn = base_normalize(c)
        if cn:
            norms.append(cn)
    norms = sorted(set(norms), key=len, reverse=True)
    return [re.compile(flex_pat(cn), flags=re.IGNORECASE) for cn in norms]

def remove_countries(text: str, country_list=None) -> str:
    if not text or not country_list:
        return text
    regs = _compile_country_regexes(tuple(country_list))
    out = text
    for rgx in regs:
        out = rgx.sub(" ", out)
    return re.sub(r"\s+", " ", out).strip()

def clean_name(name, country_list=None):
    if pd.isna(name):
        return ""
    s = base_normalize(name)
    s = strip_suffixes(s)
    s = remove_suffix_tokens(s)
    if country_list:
        s = remove_countries(s, country_list)
    return re.sub(r"\s+", " ", s).strip()

def remove_parentheses(text):
    if pd.isna(text):
        return ""
    return re.sub(r"[\(\[{].*?[\)\]}]", "", str(text)).strip()

# ==== FUNÇÕES DE TOKENIZAÇÃO ====
EXTRA_STOP = set(CORPORATE_SUFFIXES + NON_DISTINCTIVE_WORDS)

def tokenize(s: str):
    if not s:
        return set()
    toks = re.split(r"\s+", s)
    return {t for t in toks if len(t) >= 3 and t not in EXTRA_STOP}

def generate_candidate_idx(target_clean: str, subs_clean_list, subs_tokens_list):
    targets = tokenize(target_clean)
    cand_idx = []
    if not targets:
        return cand_idx
    tgt_str = target_clean
    for i, cand in enumerate(subs_clean_list):
        if not cand:
            continue
        ctoks = subs_tokens_list[i]
        if targets.intersection(ctoks):
            cand_idx.append(i)
        elif len(tgt_str) >= 5 and (tgt_str in cand or cand in tgt_str):
            cand_idx.append(i)
    return cand_idx

# ==== FUNÇÕES DE SIMILARIDADE ====
def robust_scores(a: str, b: str):
    if USING == "rapidfuzz":
        ts_set = fuzz.token_set_ratio(a, b)
        ts_sort = fuzz.token_sort_ratio(a, b)
        part = fuzz.partial_ratio(a, b)
        if distance is not None:
            try:
                jw = distance.JaroWinkler.normalized_similarity(a, b)
            except Exception:
                jw = ts_set / 100.0
            jw = jw * 100 if jw <= 1.0 else jw
        else:
            jw = ts_set
    else:
        ts_set = fuzz.token_set_ratio(a, b)
        ts_sort = fuzz.token_sort_ratio(a, b)
        part = fuzz.partial_ratio(a, b)
        jw = ts_set
    best = max(ts_set, ts_sort, part)
    return best, jw, ts_set, ts_sort, part

def cosine_ok(a: str, b: str):
    if not HAVE_SKLEARN:
        return None
    vec = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5))
    X = vec.fit_transform([a, b])
    cos = float(cosine_similarity(X[0], X[1])[0,0])
    return cos

def accept_pair(a: str, b: str):
    best, jw, ts_set, ts_sort, part = robust_scores(a, b)
    if best < MATCH_THRESHOLD:
        return False, best, jw, None
    if jw < JW_MIN:
        cos = cosine_ok(a, b)
        if cos is None or cos < COSINE_MIN:
            return False, best, jw, cos
        return True, best, jw, cos
    else:
        cos = cosine_ok(a, b)
        if cos is not None and cos < 0.60 and best < (MATCH_THRESHOLD + 5):
            return False, best, jw, cos
        return True, best, jw, cos

# ==== FUNÇÃO DE BUSCA CONTEXTUAL INTELIGENTE ====
def busca_contextual_inteligente(texto, priority_words):
    """
    Função ESPECÍFICA para identificar apenas subsidiárias PepsiCo reais
    com verificação rigorosa para evitar falsos positivos
    """
    if pd.isna(texto):
        return False, None

    texto_normalizado = unidecode.unidecode(str(texto).lower())

    # 1. Exclusão rigorosa de entidades não-PepsiCo
    for palavra in PALAVRAS_EXCLUSAO_CONTEXTUAL:
        if re.search(rf'\b{re.escape(palavra)}\b', texto_normalizado):
            return False, None

    # 2. Verificar apenas combinações MUITO ESPECÍFICAS da PepsiCo
    combinacoes_pepsico_especificas = [
        ("pepsi", "comercializadora"), ("pepsico", "comercializadora"),
        ("pepsi", "alimentos"), ("pepsico", "alimentos"),
        ("pepsi", "bebidas"), ("pepsico", "bebidas"),
        ("pepsi", "snacks"), ("pepsico", "snacks"),
        ("pepsi", "distribuidora"), ("pepsico", "distribuidora"),
        ("refrescos", "pepsi"), ("bebidas", "pepsi"),
        ("productos", "pepsi"), ("alimentos", "pepsi"),
        ("comercializadora", "nacional", "pepsi"),
        ("comercializadora", "nacional", "pepsico"),
        ("corporativo", "internacional", "mexicano", "pepsico"),
        ("latin", "american", "holdings", "pepsico"),
        ("snacks", "america", "latina", "pepsico"),
        ("bebidas", "sudamerica", "pepsico")
    ]

    for combinacao in combinacoes_pepsico_especificas:
        if all(re.search(rf'\b{re.escape(p)}\b', texto_normalizado) for p in combinacao):
            return True, f"Combinação: {combinacao}"

    # 3. Verificar apenas marcas MUITO ESPECÍFICAS da PepsiCo (exatas)
    marcas_pepsico_exatas = [
        "pepsico", "pepsi-cola", "pepsicola", "sabritas", "gamesa",
        "quaker", "gatorade", "ruffles", "cheetos", "doritos",
        "lays", "frito-lay", "mirinda", "7up", "sonrics", "emmanuel",
        "barcel", "tostitos", "tropicana", "manaos", "alvalle", "lucas"
    ]

    # Busca EXATA das marcas (não substring)
    for marca in marcas_pepsico_exatas:
        if re.search(rf'\b{re.escape(marca)}\b', texto_normalizado):
            return True, f"Marca: {marca}"

    # 4. Verificar múltiplas palavras-chave no contexto (pelo menos 2)
    palavras_encontradas = []
    for keyword in priority_words:
        keyword_normalized = unidecode.unidecode(keyword.lower())
        # Buscar palavra exata, não substring
        if re.search(rf'\b{re.escape(keyword_normalized)}\b', texto_normalizado):
            palavras_encontradas.append(keyword_normalized)
            if len(palavras_encontradas) >= 2:
                return True, f"Palavras: {', '.join(palavras_encontradas)}"

    return False, None

# ==== BUSCA PRIORITÁRIA COMPLETA EM CAMADAS ====
def busca_prioritaria_completa_camadas(list_mapping, priority_words):
    """
    Busca TODAS as palavras-chave prioritárias em camadas hierárquicas
    Retorna TODOS os matches encontrados em cada camada
    """
    priority_results = []
    linhas_processadas = set()

    print("🔍 ETAPA 0.1 - Busca prioritária COMPLETA em Global Ultimate Name...")
    if "Global Ultimate Name" in list_mapping.columns:
        for idx, name in enumerate(list_mapping["Global Ultimate Name"]):
            if idx not in linhas_processadas:
                resultado, palavra_match = busca_contextual_inteligente(name, priority_words)
                if resultado:
                    priority_results.append({
                        'lm_idx': idx,
                        'coluna': 'Global Ultimate Name',
                        'valor': name,
                        'stage': 'Priority_Global',
                        'palavra_match': palavra_match
                    })
                    linhas_processadas.add(idx)

    print("🔍 ETAPA 0.2 - Busca prioritária COMPLETA em Domestic Ultimate Name...")
    if "Domestic Ultimate Name" in list_mapping.columns:
        for idx, name in enumerate(list_mapping["Domestic Ultimate Name"]):
            if idx not in linhas_processadas:
                resultado, palavra_match = busca_contextual_inteligente(name, priority_words)
                if resultado:
                    priority_results.append({
                        'lm_idx': idx,
                        'coluna': 'Domestic Ultimate Name',
                        'valor': name,
                        'stage': 'Priority_Domestic',
                        'palavra_match': palavra_match
                    })
                    linhas_processadas.add(idx)

    print("🔍 ETAPA 0.3 - Busca prioritária COMPLETA em Account Name...")
    if "Account Name" in list_mapping.columns:
        for idx, name in enumerate(list_mapping["Account Name"]):
            if idx not in linhas_processadas:
                resultado, palavra_match = busca_contextual_inteligente(name, priority_words)
                if resultado:
                    priority_results.append({
                        'lm_idx': idx,
                        'coluna': 'Account Name',
                        'valor': name,
                        'stage': 'Priority_Account',
                        'palavra_match': palavra_match
                    })
                    linhas_processadas.add(idx)

    return priority_results, linhas_processadas

# ==== FUNÇÃO DE BUSCA PRIORITÁRIA NO ALL PRODUCTS ====
def busca_prioritaria_all_products(all_products_df, priority_words):
    """
    Busca palavras-chave prioritárias no All Products
    """
    priority_results = []

    if all_products_df.empty:
        return priority_results

    print("🔍 ETAPA 0.4 - Busca prioritária COMPLETA em All Products...")

    # Verificar coluna Account Name no All Products
    account_col = None
    for col in all_products_df.columns:
        if 'account' in str(col).lower() and 'name' in str(col).lower():
            account_col = col
            break

    if account_col:
        for idx, name in enumerate(all_products_df[account_col]):
            resultado, palavra_match = busca_contextual_inteligente(name, priority_words)
            if resultado:
                priority_results.append({
                    'ap_idx': idx,
                    'coluna': account_col,
                    'valor': name,
                    'stage': 'Priority_AllProducts',
                    'palavra_match': palavra_match
                })

    return priority_results

# ==== INTEGRAR ALL PRODUCTS PRIORITÁRIOS COM LIST MAPPING ====
def integrar_ap_com_lm(priority_ap_df, list_mapping):
    """
    Integra matches prioritários do All Products com List Mapping
    """
    integrated_results = []

    if priority_ap_df.empty or list_mapping.empty:
        return integrated_results

    # Criar mapeamento de Account Name para facilitar busca
    lm_account_map = {}
    for idx, row in list_mapping.iterrows():
        account_name = str(row.get('Account Name', '')).strip()
        if account_name:
            lm_account_map[account_name] = idx

    for _, ap_row in priority_ap_df.iterrows():
        account_name_ap = str(ap_row.get('Account_Name_AP', '')).strip()

        if account_name_ap in lm_account_map:
            lm_idx = lm_account_map[account_name_ap]
            lm_row = list_mapping.iloc[lm_idx]

            # Adicionar todos os dados relevantes
            integrated_results.append({
                'lm_idx': lm_idx,
                'ap_idx': ap_row['ap_idx'],
                'Account_Name_LM': lm_row.get('Account Name', ''),
                'Account_Name_AP': account_name_ap,
                'Global_Ultimate_Name': lm_row.get('Global Ultimate Name', ''),
                'Domestic_Ultimate_Name': lm_row.get('Domestic Ultimate Name', ''),
                'Product_Code_AP': ap_row.get('Product_Code_AP', ''),
                'Product_Description_AP': ap_row.get('Product_Description_AP', ''),
                'Sales_AP': ap_row.get('Sales_AP', ''),
                'stage': ap_row['stage'],
                'palavra_match': ap_row['palavra_match']
            })

# ==== CARREGAR DADOS ====
print("Carregando dados...")

file_business_group = "/media/Arquivos_ULA/Business_Group.xlsx"
file_listmap_path = "/media/Arquivos_ULA/List_Mapping.xlsb"
file_allprod_path = "/media/Arquivos_ULA/All_Products.xlsb"

def read_excel_smart(path, sheet_name=None, usecols=None, header=0):
    engine = "pyxlsb" if str(path).lower().endswith(".xlsb") else None
    try:
        if engine:
            return pd.read_excel(path, sheet_name=sheet_name, usecols=usecols, header=header, engine=engine)
        else:
            return pd.read_excel(path, sheet_name=sheet_name, usecols=usecols, header=header)
    except Exception as e:
        print(f"Erro ao carregar {sheet_name}: {e}")
        # Tentar descobrir os nomes das planilhas disponíveis
        try:
            if engine:
                xl = pd.ExcelFile(path, engine=engine)
            else:
                xl = pd.ExcelFile(path)
            print(f"Planilhas disponíveis em {path}: {xl.sheet_names}")
            xl.close()
        except Exception:
            print(f"Não foi possível ler as planilhas de {path}")
        return pd.DataFrame()

# Carregar Business Group
subs_df = read_excel_smart(file_business_group, sheet_name="Subs_Country")
if subs_df.empty:
    # Tentar carregar sem especificar sheet_name
    subs_df = read_excel_smart(file_business_group)
    if not subs_df.empty:
        print("Business Group carregado (planilha padrão)")
    else:
        # Criar DataFrame vazio para continuar
        subs_df = pd.DataFrame(columns=["Subsidiary List", "Country", "Region"])
        print("Business Group não pôde ser carregado, usando DataFrame vazio")

subs_df.columns = subs_df.columns.str.strip()
country_list = subs_df["Country"].astype(str).str.strip().dropna().unique().tolist() if not subs_df.empty else []

if not subs_df.empty:
    subs_df["Subs_clean"] = subs_df["Subsidiary List"].apply(lambda x: clean_name(x, country_list))
    subs_lad = subs_df[subs_df["Region"].astype(str).str.upper() == "LAD"].copy()
    subs_all = subs_df.copy()
else:
    # Criar DataFrames vazios
    subs_lad = pd.DataFrame(columns=["Subs_clean", "Subsidiary List", "Country"])
    subs_all = pd.DataFrame(columns=["Subs_clean", "Subsidiary List", "Country"])

# Preparar listas de subsidiárias
subs_lad_clean_list = subs_lad["Subs_clean"].tolist() if not subs_lad.empty else []
subs_lad_orig_list = subs_lad["Subsidiary List"].tolist() if not subs_lad.empty else []
subs_lad_tokens = [tokenize(s) for s in subs_lad_clean_list] if subs_lad_clean_list else []

subs_all_clean_list = subs_all["Subs_clean"].tolist() if not subs_all.empty else []
subs_all_orig_list = subs_all["Subsidiary List"].tolist() if not subs_all.empty else []
subs_all_tokens = [tokenize(s) for s in subs_all_clean_list] if subs_all_clean_list else []

# Carregar List Mapping
list_mapping = read_excel_smart(file_listmap_path, sheet_name="List Mapping", header=1, usecols="A:J")
if list_mapping.empty:
    # Tentar carregar sem header específico
    list_mapping = read_excel_smart(file_listmap_path, usecols="A:J")
    if not list_mapping.empty:
        print("List Mapping carregado (sem header específico)")
    else:
        # Criar DataFrame vazio para continuar
        list_mapping = pd.DataFrame(columns=["Global Ultimate Name", "Domestic Ultimate Name", "Account Name"])
        print("List Mapping não pôde ser carregado, usando DataFrame vazio")

cols_AJ = list_mapping.columns.tolist() if not list_mapping.empty else []

# Limpar colunas do List Mapping
if not list_mapping.empty:
    list_mapping["Global_clean"] = list_mapping["Global Ultimate Name"].apply(lambda x: clean_name(x, country_list))
    list_mapping["Domestic_clean"] = list_mapping["Domestic Ultimate Name"].apply(lambda x: clean_name(x, country_list))
    list_mapping["Account_clean"] = list_mapping["Account Name"].apply(lambda x: clean_name(remove_parentheses(x), country_list))
else:
    # criar Series vazias com dtype str para evitar erros de atribuição
    list_mapping["Global_clean"] = pd.Series(dtype=str)
    list_mapping["Domestic_clean"] = pd.Series(dtype=str)
    list_mapping["Account_clean"] = pd.Series(dtype=str)

# ==== CARREGAR ALL PRODUCTS COM NOME CORRETO DA PLANILHA ====
print("Carregando All Products...")
all_products = read_excel_smart(file_allprod_path, sheet_name="LAD Oppty - Revenue Line Detail", header=0)

if all_products.empty:
    print("Planilha 'LAD Oppty - Revenue Line Detail' não encontrada, procurando alternativas...")
    # Tentar encontrar a planilha correta
    all_products_sheets = []
    try:
        xl = pd.ExcelFile(file_allprod_path, engine="pyxlsb")
        all_products_sheets = xl.sheet_names
        xl.close()
        print(f"Planilhas disponíveis em All Products: {all_products_sheets}")

        # Tentar encontrar planilha com nome similar
        for sheet in all_products_sheets:
            if "revenue" in sheet.lower() or "oppty" in sheet.lower() or "lad" in sheet.lower():
                print(f" Tentando planilha: {sheet}")
                all_products = read_excel_smart(file_allprod_path, sheet_name=sheet, header=0)
                if not all_products.empty:
                    print(f"All Products carregado da planilha: {sheet}")
                    break
    except Exception as e:
        print(f"Não foi possível ler All Products: {e}")
        all_products = pd.DataFrame()

if all_products.empty:
    print("All Products não encontrado, continuando sem integração")
    account_name_map = {}
else:
    print(f"All Products carregado: {len(all_products)} linhas")
    # Criar mapeamento de Account Name para All Products
    account_name_map = {}
    for idx, row in all_products.iterrows():
        account_name = str(row.get('Account Name', '')).strip()
        if account_name:
            account_name_map[account_name] = row.to_dict()

# ==== EXECUTAR BUSCA PRIORITÁRIA COMPLETA ====
print("\n" + "="*80)
print("INICIANDO BUSCA PRIORITÁRIA COMPLETA EM CAMADAS")
print("="*80)

if not list_mapping.empty:
    priority_results, linhas_prioritarias = busca_prioritaria_completa_camadas(list_mapping, PALAVRAS_CHAVE_PRIORITARIAS)
    priority_df = pd.DataFrame(priority_results)
    print(f" {len(priority_df)} matches prioritários encontrados (TODOS!)")

    if not priority_df.empty:
        print("\nDistribuição por camada:")
        print(priority_df['stage'].value_counts())
        display(priority_df.head(20))
    else:
        print("Nenhum match prioritário encontrado")
else:
    print("List Mapping vazio, pulando busca prioritária")
    priority_df = pd.DataFrame()
    linhas_prioritarias = set()

# ==== EXECUTAR BUSCA PRIORITÁRIA NO ALL PRODUCTS ====
print("\n" + "="*80)
print("INICIANDO BUSCA PRIORITÁRIA NO ALL PRODUCTS")
print("="*80)

if not all_products.empty:
    priority_results_ap = busca_prioritaria_all_products(all_products, PALAVRAS_CHAVE_PRIORITARIAS)
    priority_ap_df = pd.DataFrame(priority_results_ap)
    print(f" {len(priority_ap_df)} matches prioritários encontrados no All Products")

    if not priority_ap_df.empty:
        print("\n Matches prioritários no All Products:")
        display(priority_ap_df.head(20))

        # Integrar com dados completos do All Products
        priority_ap_complete = []
        for _, row in priority_ap_df.iterrows():
            ap_idx = row['ap_idx']
            ap_row = all_products.iloc[ap_idx]

            priority_ap_complete.append({
                'ap_idx': ap_idx,
                'Account_Name_AP': ap_row.get('Account Name', ''),
                'Product_Code_AP': ap_row.get('Product Code', ''),
                'Product_Description_AP': ap_row.get('Product Description', ''),
                'Sales_AP': ap_row.get('Sales', ''),
                'Quantity_AP': ap_row.get('Quantity', ''),
                'Customer_Segment_AP': ap_row.get('Customer Segment', ''),
                'stage': row['stage'],
                'palavra_match': row['palavra_match']
            })

        priority_ap_final_df = pd.DataFrame(priority_ap_complete)
        print(f" {len(priority_ap_final_df)} registros completos do All Products")

    else:
        print(" Nenhum match prioritário encontrado no All Products")
        priority_ap_final_df = pd.DataFrame()
else:
    print(" All Products vazio, pulando busca prioritária")
    priority_ap_final_df = pd.DataFrame()

# ==== INTEGRAR ALL PRODUCTS PRIORITÁRIOS COM LIST MAPPING ====
if not priority_ap_final_df.empty and not list_mapping.empty:
    integrated_ap_lm = integrar_ap_com_lm(priority_ap_final_df, list_mapping)
    integrated_ap_lm_df = pd.DataFrame(integrated_ap_lm)
    print(f"{len(integrated_ap_lm_df)} registros integrados (All Products + List Mapping)")

    if not integrated_ap_lm_df.empty:
        display(integrated_ap_lm_df.head(10))
else:
    integrated_ap_lm_df = pd.DataFrame()

# ==== ENCONTRAR SUBSIDIÁRIAS PARA MATCHES PRIORITÁRIOS ====
print("\nEncontrando subsidiárias para matches prioritários...")

if (not subs_all.empty) and (not priority_df.empty):
    subs_pais_map = dict(zip(subs_all["Subsidiary List"], subs_all["Country"])) if not subs_all.empty else {}
    priority_com_subs = []

    for _, row in priority_df.iterrows():
        lm_idx = row['lm_idx']
        lm_row = list_mapping.iloc[lm_idx]

        # Buscar TODAS as subsidiárias correspondentes
        target_clean = lm_row.get("Global_clean") or lm_row.get("Domestic_clean") or lm_row.get("Account_clean") or ""

        matches_subs = []
        for subs_idx, subs_clean in enumerate(subs_all_clean_list):
            if not subs_clean:
                continue
            score = fuzz.token_set_ratio(target_clean, subs_clean)
            if score >= MATCH_THRESHOLD:
                subs_name = subs_all_orig_list[subs_idx]
                matches_subs.append({
                    'subsidiary': subs_name,
                    'score': score,
                    'pais': subs_pais_map.get(subs_name, 'Desconhecido')
                })

        # Ordenar por score e pegar as melhores
        matches_subs.sort(key=lambda x: x['score'], reverse=True)

        for match in matches_subs[:5]:  # Top 5 matches por linha
            # Criar um dicionário com TODAS as colunas de A:J
            lm_data = {}
            for col in cols_AJ:
                lm_data[f'lm_{col}'] = lm_row.get(col, None)

            priority_com_subs.append({
                'lm_idx': lm_idx,
                **lm_data,  # Incluir todas as colunas de A:J
                'subsidiary': match['subsidiary'],
                'subs_country': match['pais'],
                'score': match['score'],
                'match_type': 'PRIORITY_KEYWORD',
                'stage': row['stage'],
                'palavra_match': row['palavra_match']
            })

    priority_final_df = pd.DataFrame(priority_com_subs)
    print(f" {len(priority_final_df)} matches prioritários com subsidiárias encontrados")

    # Mostrar as colunas de A:J na exibição
    if not priority_final_df.empty:
        print("\n Exibindo matches prioritários com todas as colunas A:J:")
        display_cols = ['lm_idx', 'stage', 'palavra_match'] + [f'lm_{col}' for col in cols_AJ] + ['subsidiary', 'subs_country', 'score']
        display(priority_final_df[display_cols].head(20))
else:
    print(" Dados insuficientes para buscar subsidiárias prioritárias")
    priority_final_df = pd.DataFrame()

# ==== INTEGRAÇÃO COM ALL PRODUCTS ====
print("\nINTEGRANDO COM ALL PRODUCTS...")

if not priority_final_df.empty and account_name_map:
    # Adicionar dados do All Products aos matches prioritários
    priority_with_all_products = []
    for _, row in priority_final_df.iterrows():
        account_name = str(row.get('lm_Account Name', '')).strip()
        all_products_data = account_name_map.get(account_name, {})

        new_row = row.to_dict()
        # Adicionar campos relevantes do All Products
        for field in ['Product Code', 'Product Description', 'Sales', 'Quantity', 'Customer Segment']:
            new_row[f'all_products_{field}'] = all_products_data.get(field, 'N/A')

        priority_with_all_products.append(new_row)

    priority_final_df = pd.DataFrame(priority_with_all_products)
    print(f"Integração com All Products concluída: {len(priority_final_df)} registros")
else:
    print(" Não há dados para integrar com All Products")

# ==== FUNÇÃO DE MATCHING PARA ETAPAS NORMAIS ====
def match_LM_column_to_subs(list_series, subs_clean_list, subs_orig_list, subs_tokens_list, excluded_indices=None):
    """Função de matching com suporte para excluir índices já utilizados"""
    if excluded_indices is None:
        excluded_indices = set()

    matches_map = {}
    used_subs = set()

    for lm_idx, target in list_series.items():
        if lm_idx in excluded_indices:
            continue
        t = str(target or "").strip()
        if not t:
            continue

        cand_idx = generate_candidate_idx(t, subs_clean_list, subs_tokens_list)

        if not cand_idx:
            # proteger caso subs_clean_list vazio
            if not subs_clean_list:
                continue
            raw = process.extract(t, subs_clean_list, scorer=fuzz.token_set_ratio, limit=TOPN_PER_ROW)
            cand_idx = []
            for x in raw:
                if len(x) >= 3 and isinstance(x[2], int):
                    cand_idx.append(x[2])
                else:
                    try:
                        cand_idx.append(subs_clean_list.index(x[0]))
                    except Exception:
                        continue

        cand_idx = [ci for ci in dict.fromkeys(ci for ci in cand_idx if ci is not None)]
        accepted = []
        cand_strings = [subs_clean_list[i] for i in cand_idx] if cand_idx else []

        if cand_strings:
            raw2 = process.extract(t, cand_strings, scorer=fuzz.token_set_ratio, limit=min(TOPN_PER_ROW, len(cand_strings)))
            for entry in raw2:
                if len(entry) >= 3:
                    choice_str, prelim, pos = entry[0], entry[1], entry[2]
                else:
                    choice_str, prelim = entry[0], entry[1]
                    pos = cand_strings.index(choice_str)
                real_idx = cand_idx[pos]
                s_clean = subs_clean_list[real_idx]
                ok, best, jw, cos = accept_pair(t, s_clean)
                if ok:
                    accepted.append((subs_orig_list[real_idx], int(round(best)), float(jw), (None if cos is None else float(cos))))

        if accepted:
            matches_map[lm_idx] = accepted
            for s, _best, _jw, _cos in accepted:
                used_subs.add(s)

    return matches_map, used_subs

# ==== ETAPAS NORMAIS (apenas se houver dados) ====
if (not list_mapping.empty) and subs_lad_clean_list:
    # ==== ETAPA 1 - GLOBAL (excluindo prioritários) ====
    print("\n ETAPA 1 - List Mapping: Global VS Subsidiary (LAD)")
    matches_stage1, used_subs_1 = match_LM_column_to_subs(
        list_mapping["Global_clean"],
        subs_lad_clean_list,
        subs_lad_orig_list,
        subs_lad_tokens,
        excluded_indices=linhas_prioritarias
    )

    if matches_stage1:
        idxs1 = sorted(matches_stage1.keys())
        stage1_df = list_mapping.loc[idxs1, cols_AJ].copy()
        stage1_df.insert(0, "RowIndex", idxs1)
        stage1_df["Matched_Subsidiaries"] = [[m[0] for m in matches_stage1[i]] for i in idxs1]
        stage1_df["Best_Scores"] = [[m[1] for m in matches_stage1[i]] for i in idxs1]
        stage1_df["JW"] = [[m[2] for m in matches_stage1[i]] for i in idxs1]
        stage1_df["Cosine"] = [[m[3] for m in matches_stage1[i]] for i in idxs1]
        stage1_df["Matched_Count"] = stage1_df["Matched_Subsidiaries"].apply(len)

        # Integrar com All Products
        if account_name_map:
            stage1_with_products = []
            for _, row in stage1_df.iterrows():
                account_name = str(row.get('Account Name', '')).strip()
                all_products_data = account_name_map.get(account_name, {})

                new_row = row.to_dict()
                for field in ['Product Code', 'Product Description', 'Sales', 'Quantity', 'Customer Segment']:
                    new_row[f'all_products_{field}'] = all_products_data.get(field, 'N/A')
                stage1_with_products.append(new_row)

            stage1_df = pd.DataFrame(stage1_with_products)
        print(f"Etapa 1: {len(stage1_df)} linhas encontradas")
    else:
        stage1_df = pd.DataFrame(columns=["RowIndex"] + cols_AJ + ["Matched_Subsidiaries", "Best_Scores", "JW", "Cosine", "Matched_Count"])
        print(" Nenhuma linha encontrada na Etapa 1")

    matched_idx_1 = set(matches_stage1.keys() if matches_stage1 else [])
    used_subs_all = set(used_subs_1 if matches_stage1 else set())

    # ==== ETAPA 2 - DOMESTIC (excluindo prioritários + etapa 1) ====
    print("\nETAPA 2 - List Mapping: Domestic VS Subsidiary (ALL)")
    remaining_indices = matched_idx_1.union(linhas_prioritarias)
    remaining_dom = list_mapping.drop(index=remaining_indices, errors="ignore")

    if not subs_all.empty:
        subs2_mask = ~subs_all["Subsidiary List"].isin(used_subs_all)
        subs2_clean = subs_all.loc[subs2_mask, "Subs_clean"].tolist()
        subs2_orig = subs_all.loc[subs2_mask, "Subsidiary List"].tolist()
        subs2_tokens = [tokenize(s) for s in subs2_clean]
    else:
        subs2_clean, subs2_orig, subs2_tokens = [], [], []

    matches_stage2, used_subs_2 = match_LM_column_to_subs(
        remaining_dom["Domestic_clean"],
        subs2_clean,
        subs2_orig,
        subs2_tokens,
        excluded_indices=linhas_prioritarias
    )

    if matches_stage2:
        idxs2 = sorted(matches_stage2.keys())
        stage2_df = list_mapping.loc[idxs2, cols_AJ].copy()
        stage2_df.insert(0, "RowIndex", idxs2)
        stage2_df["Matched_Subsidiaries"] = [[m[0] for m in matches_stage2[i]] for i in idxs2]
        stage2_df["Best_Scores"] = [[m[1] for m in matches_stage2[i]] for i in idxs2]
        stage2_df["JW"] = [[m[2] for m in matches_stage2[i]] for i in idxs2]
        stage2_df["Cosine"] = [[m[3] for m in matches_stage2[i]] for i in idxs2]
        stage2_df["Matched_Count"] = stage2_df["Matched_Subsidiaries"].apply(len)

        # Integrar com All Products
        if account_name_map:
            stage2_with_products = []
            for _, row in stage2_df.iterrows():
                account_name = str(row.get('Account Name', '')).strip()
                all_products_data = account_name_map.get(account_name, {})

                new_row = row.to_dict()
                for field in ['Product Code', 'Product Description', 'Sales', 'Quantity', 'Customer Segment']:
                    new_row[f'all_products_{field}'] = all_products_data.get(field, 'N/A')
                stage2_with_products.append(new_row)

            stage2_df = pd.DataFrame(stage2_with_products)
        print(f"Etapa 2: {len(stage2_df)} linhas encontradas")
    else:
        stage2_df = pd.DataFrame(columns=["RowIndex"] + cols_AJ + ["Matched_Subsidiaries", "Best_Scores", "JW", "Cosine", "Matched_Count"])
        print("Nenhuma linha encontrada na Etapa 2")

    matched_idx_2 = set(matches_stage2.keys() if matches_stage2 else [])
    used_subs_all.update(used_subs_2 if matches_stage2 else set())

# ==== ETAPA 3 - ACCOUNT (excluindo prioritários + etapas 1+2) ====
print("\n ETAPA 3 - List Mapping: Account VS Subsidiary (ALL)")
remaining_indices = matched_idx_1.union(matched_idx_2).union(linhas_prioritarias)
remaining_acc = list_mapping.drop(index=remaining_indices, errors="ignore")

if not subs_all.empty:
    subs3_mask = ~subs_all["Subsidiary List"].isin(used_subs_all)
    subs3_clean = subs_all.loc[subs3_mask, "Subs_clean"].tolist()
    subs3_orig = subs_all.loc[subs3_mask, "Subsidiary List"].tolist()
    subs3_tokens = [tokenize(s) for s in subs3_clean]
else:
    subs3_clean, subs3_orig, subs3_tokens = [], [], []

matches_stage3, used_subs_3 = match_LM_column_to_subs(
    remaining_acc["Account_clean"],
    subs3_clean,
    subs3_orig,
    subs3_tokens,
    excluded_indices=linhas_prioritarias
)

if matches_stage3:
    idxs3 = sorted(matches_stage3.keys())
    stage3_df = list_mapping.loc[idxs3, cols_AJ].copy()
    stage3_df.insert(0, "RowIndex", idxs3)
    stage3_df["Matched_Subsidiaries"] = [[m[0] for m in matches_stage3[i]] for i in idxs3]
    stage3_df["Best_Scores"] = [[m[1] for m in matches_stage3[i]] for i in idxs3]
    stage3_df["JW"] = [[m[2] for m in matches_stage3[i]] for i in idxs3]
    stage3_df["Cosine"] = [[m[3] for m in matches_stage3[i]] for i in idxs3]
    stage3_df["Matched_Count"] = stage3_df["Matched_Subsidiaries"].apply(len)

# Integrar com All Products
    if account_name_map:
        stage3_with_products = []
        for _, row in stage3_df.iterrows():
            account_name = str(row.get('Account Name', '')).strip()
            all_products_data = account_name_map.get(account_name, {})

            new_row = row.to_dict()
            for field in ['Product Code', 'Product Description', 'Sales', 'Quantity', 'Customer Segment']:
                new_row[f'all_products_{field}'] = all_products_data.get(field, 'N/A')
            stage3_with_products.append(new_row)

        stage3_df = pd.DataFrame(stage3_with_products)
    print(f"Etapa 3: {len(stage3_df)} linhas encontradas")
else:
    stage3_df = pd.DataFrame(columns=["RowIndex"] + cols_AJ + ["Matched_Subsidiaries", "Best_Scores", "JW", "Cosine", "Matched_Count"])
    print("Nenhuma linha encontrada na Etapa 3")

# Verificação de dados insuficientes (se necessário)
if not list_mapping.empty and not subs_lad_clean_list:
    print("Dados insuficientes para etapas normais")
    stage1_df = pd.DataFrame()
    stage2_df = pd.DataFrame()
    stage3_df = pd.DataFrame()

# ==== RESUMO FINAL ====
print("\n" + "="*80)
print("RESUMO FINAL - TODAS AS ETAPAS")
print("="*80)
# Preparar resultados das etapas normais para incluir todas as colunas A:J
def prepare_stage_df_with_all_columns(stage_df, stage_name):
    if stage_df.empty:
        return pd.DataFrame()

    # Garantir que temos todas as colunas de A:J
    result_df = stage_df.copy()
    for col in cols_AJ:
        if col not in result_df.columns:
            # usar valores da list_mapping (quando RowIndex existe)
            try:
                result_df[col] = list_mapping[col].iloc[result_df['RowIndex']].values
            except Exception:
                result_df[col] = None

    result_df['Stage'] = stage_name
    return result_df

stage1_df_full = prepare_stage_df_with_all_columns(stage1_df, "Global")
stage2_df_full = prepare_stage_df_with_all_columns(stage2_df, "Domestic")
stage3_df_full = prepare_stage_df_with_all_columns(stage3_df, "Account")

# Adicionar colunas de top match
for _df in [stage1_df_full, stage2_df_full, stage3_df_full]:
    if not _df.empty:
        _df["Top_Subsidiary"] = _df["Matched_Subsidiaries"].apply(lambda xs: xs[0] if isinstance(xs, list) and len(xs) > 0 else None)
        _df["Top_Score"] = _df["Best_Scores"].apply(lambda xs: xs[0] if isinstance(xs, list) and len(xs) > 0 else None)

# Combinar todos os resultados
resumo_final = pd.concat([priority_final_df, stage1_df_full, stage2_df_full, stage3_df_full], ignore_index=True, sort=False)

print(f" Resumo completo: {len(resumo_final)} linhas totais")
print(f"   - Priority: {len(priority_final_df)}")
print(f"   - Global: {len(stage1_df_full)}")
print(f"   - Domestic: {len(stage2_df_full)}")
print(f"   - Account: {len(stage3_df_full)}")

if not resumo_final.empty:
    # Mostrar colunas principais para visualização
    display_cols = ['Stage', 'lm_Global Ultimate Name', 'lm_Domestic Ultimate Name', 'lm_Account Name',
                   'subsidiary', 'subs_country', 'score', 'palavra_match', 'Top_Subsidiary', 'Top_Score']
    display_cols = [col for col in display_cols if col in resumo_final.columns]
    display(resumo_final[display_cols].head(20))
else:
    print("Nenhum resultado encontrado")

# =========================
# ETAPA 4 - ALL PRODUCTS (cruzamento)
# =========================
print("\n" + "="*80)
print(" ETAPA 4 - CRUZAMENTO COM ALL PRODUCTS")
print("="*80)

def detect_col(df, regex_list):
    if df is None or df.columns is None:
        return None
    cols = list(df.columns)
    norm = {c: base_normalize(c) for c in cols}
    for rgx in regex_list:
        try:
            pat = re.compile(rgx, flags=re.I)
        except Exception:
            continue
        for c in cols:
            try:
                if pat.search(str(c)) or pat.search(norm[c]):
                    return c
            except Exception:
                continue
    return cols[0] if cols else None

# Detectar colunas no All Products (regexes mais robustas)
ap_customer_col = detect_col(all_products, [
    r"\bcustomer\s*name\b",
    r"\bcust(?:omer)?\s*name\b",
    r"\bclient\s*name\b",
    r"\baccount\s*name\b",
])

ap_registry_col = detect_col(all_products, [
    r"\bcustomer\b.*\bregistry\b.*\bid\b",
    r"\bregistry\b.*\bid\b",
    r"\bcustomer\b.*\bid\b",
    r"\bclient\b.*\bid\b",
    r"\baccount\b.*\bid\b",
])

# Preparar dados do All Products
if not all_products.empty:
    ap = all_products.copy()
    if ap_customer_col:
        ap.rename(columns={ap_customer_col: "AP_Customer_Name"}, inplace=True)
    else:
        ap["AP_Customer_Name"] = ap.index.astype(str)

    if ap_registry_col:
        ap.rename(columns={ap_registry_col: "AP_Customer_Registry_ID"}, inplace=True)

    ap["AP_Customer_Name"] = ap["AP_Customer_Name"].astype(str)
    ap["Customer_clean"] = ap["AP_Customer_Name"].apply(lambda x: clean_name(x, country_list))
    ap_clean_list = ap["Customer_clean"].tolist()
    ap_tokens_list = [tokenize(s) for s in ap_clean_list]

    print(f"All Products preparado: {len(ap)} linhas")
else:
    ap = pd.DataFrame()
    ap_clean_list = []
    ap_tokens_list = []
    print("All Products vazio, pulando etapa 4")

# Função para encontrar matches por nome
def name_matches_from_clean(target_clean):
    ap_idx = []
    if not target_clean or not ap_clean_list:
        return ap_idx

    cand_idx = generate_candidate_idx(target_clean, ap_clean_list, ap_tokens_list)

    if not cand_idx:
        raw = process.extract(target_clean, ap_clean_list, scorer=fuzz.token_set_ratio, limit=TOPN_PER_ROW)
        cand_idx = []
        for x in raw:
            if len(x) >= 3 and isinstance(x[2], int):
                cand_idx.append(x[2])
            else:
                try:
                    cand_idx.append(ap_clean_list.index(x[0]))
                except Exception:
                    continue

    cand_idx = [ci for ci in dict.fromkeys(ci for ci in cand_idx if ci is not None)]
    out = []

    if cand_idx:
        cand_strings = [ap_clean_list[i] for i in cand_idx]
        raw2 = process.extract(target_clean, cand_strings, scorer=fuzz.token_set_ratio, limit=min(TOPN_PER_ROW, len(cand_strings)))
        for entry in raw2:
            if len(entry) >= 3:
                _choice_str, _pre, pos = entry[0], entry[1], entry[2]
            else:
                _choice_str, _pre = entry[0], entry[1]
                pos = cand_strings.index(_choice_str)
            real_idx = cand_idx[pos]
            s_clean = ap_clean_list[real_idx]
            ok, best, jw, cos = accept_pair(target_clean, s_clean)
            if ok:
                out.append((real_idx, int(round(best)), float(jw), (None if cos is None else float(cos))))
    return out

# Função para encontrar matches por ID
def exact_id_matches_from_values(id_value):
    if id_value is None or (isinstance(id_value, float) and pd.isna(id_value)):
        return []
    if "AP_Customer_Registry_ID" not in ap.columns:
        return []

    key = str(id_value).strip()
    return list(ap.index[ap["AP_Customer_Registry_ID"].astype(str).str.strip() == key])

# Construir matches para cada estágio
def build_ap_matches_for_stage(stage_df, stage_label, lm_key_col_name):
    rows = []
    if stage_df is None or stage_df.empty:
        return pd.DataFrame(rows)

    for _, row in stage_df.iterrows():
        lm_idx = row.get("RowIndex")
        if lm_idx is None:
            continue
        target_clean = list_mapping.loc[lm_idx, lm_key_col_name]

        # Buscar matches por nome
        by_name = name_matches_from_clean(target_clean)

        # Buscar matches por ID
        lm_ids = []
        for c in list_mapping.columns:
            if "id" in str(c).lower():
                lm_ids.append(list_mapping.loc[lm_idx, c])

        by_id = set()
        for v in lm_ids:
            for ap_idx in exact_id_matches_from_values(v):
                by_id.add(ap_idx)

        # Combinar resultados
        name_idx_set = {i for (i, _best, _jw, _cos) in by_name}
        both = name_idx_set.intersection(by_id)
        only_name = name_idx_set - both
        only_id = by_id - both

        lm_payload = list_mapping.loc[lm_idx].to_dict()
        lm_payload_pref = {f"LM_{k}": v for k, v in lm_payload.items()}
        lm_payload_pref["Stage"] = stage_label

        # Adicionar matches
        for i in both:
            ap_payload = ap.loc[i].to_dict()
            ap_payload_pref = {f"AP_{k}" if not str(k).startswith("AP_") else k: v for k, v in ap_payload.items()}
            _best = _jw = _cos = None
            for j, b, jw, co in by_name:
                if j == i:
                    _best, _jw, _cos = b, jw, co
                    break
            rows.append({**lm_payload_pref, **ap_payload_pref, "Match_Mode": "Both", "Name_Score": _best, "Name_JW": _jw, "Name_Cosine": _cos})

        for i, b, jw, co in by_name:
            if i in both:
                continue
            ap_payload = ap.loc[i].to_dict()
            ap_payload_pref = {f"AP_{k}" if not str(k).startswith("AP_") else k: v for k, v in ap_payload.items()}
            rows.append({**lm_payload_pref, **ap_payload_pref, "Match_Mode": "Name", "Name_Score": b, "Name_JW": jw, "Name_Cosine": co})

        for i in only_id:
            ap_payload = ap.loc[i].to_dict()
            ap_payload_pref = {f"AP_{k}" if not str(k).startswith("AP_") else k: v for k, v in ap_payload.items()}
            rows.append({**lm_payload_pref, **ap_payload_pref, "Match_Mode": "ID", "Name_Score": None, "Name_JW": None, "Name_Cosine": None})

    return pd.DataFrame(rows)

# Construir matches para cada estágio
AP_Global_All = build_ap_matches_for_stage(stage1_df, "Global", "Global_clean")
AP_Domestic_All = build_ap_matches_for_stage(stage2_df, "Domestic", "Domestic_clean")
AP_Account_All = build_ap_matches_for_stage(stage3_df, "Account", "Account_clean")

print("\nEtapa 4 - Matches com ALL PRODUCTS:")
print(" AP_Global_All  :", len(AP_Global_All))
print(" AP_Domestic_All:", len(AP_Domestic_All))
print(" AP_Account_All :", len(AP_Account_All))

# =========================
# ETAPA 5 - Validação de Produtos
# =========================
print("\n" + "="*80)
print(" ETAPA 5 - VALIDAÇÃO DE PRODUTOS")
print("="*80)

# Carregar produtos do Summary
try:
    bayer_summary = read_excel_smart(file_business_group, sheet_name="Summary", header=None, usecols=[1])
    bayer_summary.columns = ["Summary_ColB"]
    bayer_products_raw = bayer_summary["Summary_ColB"].dropna().astype(str)

    def clean_product(p):
        s = base_normalize(p)
        return re.sub(r"\s+", " ", s).strip()

    bayer_products = bayer_products_raw.apply(clean_product)
    bayer_products = bayer_products[bayer_products.str.len() > 0].unique().tolist()
    print(f"ℹ Produtos do Summary carregados: {len(bayer_products)} itens.")
except Exception as e:
    bayer_products = []
    print(" Não foi possível ler produtos do Summary (coluna B).", e)

# Identificar colunas de produto no All Products
def find_product_columns(ap_cols):
    pats = r"(product|produto|sku|item|material|brand|family|line|portfolio|description|descri[cç][aã]o|desc)"
    return [c for c in ap_cols if re.search(pats, str(c), flags=re.I)]

product_cols_in_ap = find_product_columns(ap.columns.tolist()) if not ap.empty else []

if not product_cols_in_ap:
    guesses = [c for c in ap.columns if re.search(r"(prod|sku|item|material|desc|brand)", str(c), re.I)] if not ap.empty else []
    product_cols_in_ap = guesses

print(f"ℹColunas de produto identificadas: {product_cols_in_ap}")

# Função para calcular score de produto
def product_best_score(a: str, b: str) -> int:
    a = str(a or ""); b = str(b or "")
    a = re.sub(r"\s+", " ", base_normalize(a)).strip()
    b = re.sub(r"\s+", " ", base_normalize(b)).strip()
    if not a or not b:
        return 0
    s1 = fuzz.token_set_ratio(a, b)
    s2 = fuzz.token_sort_ratio(a, b)
    s3 = fuzz.partial_ratio(a, b)
    return int(max(s1, s2, s3))

# Parâmetros para validação de produtos
PRODUCT_MIN = 50
PRODUCT_MAX = 60

# Validar produtos
prod_rows = []
AP_all_matches = pd.concat([AP_Global_All, AP_Domestic_All, AP_Account_All], ignore_index=True) if (((not AP_Global_All.empty) or (not AP_Domestic_All.empty) or (not AP_Account_All.empty))) else pd.DataFrame()

if (not AP_all_matches.empty) and bayer_products and product_cols_in_ap:
    for idx, r in AP_all_matches.iterrows():
        ap_cust_name = r.get("AP_AP_Customer_Name", r.get("AP_Customer_Name", None))
        ap_cust_id = r.get("AP_AP_Customer_Registry_ID", r.get("AP_Customer_Registry_ID", None))
        stage_src = r.get("Stage", None)

        for pc in product_cols_in_ap:
            ap_val = r.get(f"AP_{pc}", None)
            if ap_val is None:
                continue
            if pd.isna(ap_val) or str(ap_val).strip() == "":
                continue

            for bp in bayer_products:
                sc = product_best_score(ap_val, bp)
                if PRODUCT_MIN <= sc <= PRODUCT_MAX:
                    prod_rows.append({
                        "Stage": stage_src,
                        "AP_Customer_Name": ap_cust_name,
                        "AP_Customer_Registry_ID": ap_cust_id,
                        "AP_Product_Column": pc,
                        "AP_Product_Value": ap_val,
                        "Summary_Product": bp,
                        "Product_Score": sc
                    })

AP_Products_Validation = pd.DataFrame(prod_rows)
print(f"Etapa 5 - Validação de Produtos (50-60): {len(AP_Products_Validation)} pares produto encontrados.")

# ==== EXPORTAÇÃO FINAL ====
out_xlsx = "/media/Arquivos_ULA/RESULTADO_FINAL_COMPLETO_PEPSICO.xlsx"
try:
    with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
        if not priority_final_df.empty:
            priority_final_df.to_excel(writer, sheet_name="Stage0_Priority", index=False)
        if not stage1_df.empty:
            stage1_df.to_excel(writer, sheet_name="Stage1_Global", index=False)
        if not stage2_df.empty:
            stage2_df.to_excel(writer, sheet_name="Stage2_Domestic", index=False)
        if not stage3_df.empty:
            stage3_df.to_excel(writer, sheet_name="Stage3_Account", index=False)
        if not resumo_final.empty:
            resumo_final.to_excel(writer, sheet_name="Resumo_Completo", index=False)
        if not priority_ap_final_df.empty:
            priority_ap_final_df.to_excel(writer, sheet_name="AP_Priority_Matches", index=False)
        if not integrated_ap_lm_df.empty:
            integrated_ap_lm_df.to_excel(writer, sheet_name="AP_LM_Integrated", index=False)
        if not AP_Global_All.empty:
            AP_Global_All.to_excel(writer, sheet_name="AP_Global_All", index=False)
        if not AP_Domestic_All.empty:
            AP_Domestic_All.to_excel(writer, sheet_name="AP_Domestic_All", index=False)
        if not AP_Account_All.empty:
            AP_Account_All.to_excel(writer, sheet_name="AP_Account_All", index=False)
        if not AP_Products_Validation.empty:
            AP_Products_Validation.to_excel(writer, sheet_name="AP_Products_Validation", index=False)

    print(f"\nArquivo exportado: {out_xlsx}")
    print("Processamento concluído com sucesso!")
except Exception as e:
    print(f"Erro ao exportar arquivo: {e}")
    print("Processamento concluído com erros")

# Nova seção